In [1]:
import os
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

/Users/a06411/opt/anaconda3/envs/fluentPython/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark = (SparkSession.builder.appName('sqlite-api')
                             .config("spark.driver.host","127.0.0.1") 
                             .config("spark.driver.bindAddress","127.0.0.1")
                             .config("spark.jars","{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
                             .config("spark.driver.extraClassPath","{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
                             .getOrCreate())

23/06/09 21:37:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/09 21:37:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/09 21:37:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark

In [4]:
import sqlite3

con = sqlite3.connect("./chinook.db")

In [5]:
query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

In [6]:
pd.read_sql_query(query,con)

,name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


In [13]:
spark.sql("CREATE DATABASE IF NOT EXISTS chinook_db;")

DataFrame[]

In [15]:
spark.sql("show databases").show()

+----------+
| namespace|
+----------+
|chinook_db|
|   default|
+----------+



In [17]:
spark.sql("use chinook_db")

DataFrame[]

In [7]:
py_df_customers = spark.read.format('jdbc') \
        .options(driver='org.sqlite.JDBC', dbtable='customers',
                 url='jdbc:sqlite:{}/chinook.db'.format(os.getcwd()))\
        .load()

In [11]:
py_df_customers.printSchema()

root
 |-- CustomerId: integer (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Fax: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- SupportRepId: integer (nullable = true)



In [8]:
py_df_customers.show()

+----------+---------+-----------+--------------------+--------------------+-------------------+-----+--------------+----------+------------------+------------------+--------------------+------------+
|CustomerId|FirstName|   LastName|             Company|             Address|               City|State|       Country|PostalCode|             Phone|               Fax|               Email|SupportRepId|
+----------+---------+-----------+--------------------+--------------------+-------------------+-----+--------------+----------+------------------+------------------+--------------------+------------+
|         1|     Luís|  Gonçalves|Embraer - Empresa...|Av. Brigadeiro Fa...|São José dos Campos|   SP|        Brazil| 12227-000|+55 (12) 3923-5555|+55 (12) 3923-5566|luisg@embraer.com.br|           3|
|         2|   Leonie|     Köhler|                null|Theodor-Heuss-Str...|          Stuttgart| null|       Germany|     70174|  +49 0711 2842222|              null|leonekohler@surfe...|         

In [18]:
py_df_customers.write.mode("overwrite").saveAsTable('customers')

In [20]:
spark.sql("show tables").show()

+----------+---------+-----------+
| namespace|tableName|isTemporary|
+----------+---------+-----------+
|chinook_db|customers|      false|
+----------+---------+-----------+



In [22]:
spark.sql("desc customers").show()

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
|  CustomerId|      int|   null|
|   FirstName|   string|   null|
|    LastName|   string|   null|
|     Company|   string|   null|
|     Address|   string|   null|
|        City|   string|   null|
|       State|   string|   null|
|     Country|   string|   null|
|  PostalCode|   string|   null|
|       Phone|   string|   null|
|         Fax|   string|   null|
|       Email|   string|   null|
|SupportRepId|      int|   null|
+------------+---------+-------+



In [21]:
spark.sql("select count(*) from customers").show()

+--------+
|count(1)|
+--------+
|      59|
+--------+

